In [1]:
import pandas as pd
import numpy as np
import os
import datetime

In [2]:
SAMPLE_SIZE = 5
PORTFOLIO_SIZE = 5
STOCKS_FILEPATH = 'daily_data/'

In [3]:
# Lista de los nombres de cada archivo
stocks_list = os.listdir(STOCKS_FILEPATH)
stock_names = np.array([stock.split('-')[0] for stock in stocks_list])

In [4]:
# Crear dataframe con todas las acciones
df = pd.DataFrame()

for stock in stocks_list:
    stock_name = stock.split('-')[0]
    df[stock_name] = pd.read_parquet(os.path.join(STOCKS_FILEPATH, stock))['close']

In [5]:
# Función para hallar el número de valores NA para cada acción
def na_count(df):
    return np.array([len(df[stock_name][df[stock_name].isna()]) for stock_name in stock_names])

In [50]:
# Fecha inicial a partir de la cual se tomarán las acciones
# (teniendo en cuenta que muchas no existían en años anteriores)
initial_date = datetime.date(2014,1,1)
# Filtrar las fechas para incluir fechas posteriores a la fecha inicial
df_after_date = df[df.index > initial_date]
df_after_date.shape

(1697, 1474)

In [51]:
# Hallar el número de días con NA para cada acción
nas = na_count(df_after_date)
# Hallar las acciones que más días faltantes tienen
nas_indices = np.argsort(nas)[::-1] 

nas_indices

array([ 165, 1387,  985, ...,  288,  718,    0], dtype=int64)

In [52]:
# Hallar el número de acciones a las que les faltan más de 252 días de datos
stock_indices_to_remove = nas_indices[nas[nas_indices] > 252]
print(f'Se eliminarán {len(stock_indices_to_remove)} acciones de la lista de acciones elegibles')

Se eliminarán 730 acciones de la lista de acciones elegibles


In [54]:
# Sacar de la lista las acciones a las que les falten más de 252 días de datos
stocks_to_remove = stock_names[stock_indices_to_remove]
df_applicable_stocks = df_after_date.drop(stocks_to_remove, axis=1)
df_applicable_stocks.shape

(1697, 744)

In [59]:
# Llenar los valores NA de las acciones restantes con el precio anterior / siguiente
df_applicable_stocks_no_na = df_applicable_stocks.fillna(method='ffill').fillna(method='bfill')